In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pyautogui
import time

# Check if string is a number by converting it into a float
def is_number(s):
    try:
        float(s) 
        return True
    except ValueError:
        return False
    
# click on element until is available to be clicked 
def click_element(driver, xpath):
    return WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()

# get the number of item in the sub-cat
def get_no_of_item():
    # get total no of item in this category
    ele_no_of_item = driver.find_element(By.XPATH,'//*[@id="headroomElement"]/div/div/div/div/div[1]/div/span')
    str_no_of_item = ele_no_of_item.text
    print(f'str_no_of_item : {str_no_of_item}')
    no_of_item = int(str_no_of_item.split()[0])
    return no_of_item

# click on loadmore button to show all items in the sub-cat
def show_all_items(num_of_loadmore):
    #Find and click on 'LOAD MORE' button to show all items
        
    # XPATH for loadmore button
    if category == 'WOMEN' and sub_cat == 3:
        xpath_loadmore ='//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/nav/div/div[2]/div[2]/div[1]/a/div/span[1]'
    elif category == 'MEN' and sub_cat == 5:
        xpath_loadmore ='//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/section[4]/div/div/div[2]/div[1]/a/div/span[1]'
    else:
        xpath_loadmore ='//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/section[3]/div/div/div[2]/div[1]/a'
            
    if num_of_loadmore > 0 :
        time.sleep(2)
 
        for x in range(num_of_loadmore):
            time.sleep(1)
            pyautogui.press('esc')
            loadMoreButton = click_element(driver, xpath)
            oadMoreButton.click()

# Scrape all detail of the item such as review, category, sub-cat, name, price, description, 
# dictionary for the availability of color and size
def get_item_details():
    
    item_dict = {}
    
    #Scrape the review score
    list_of_em_tags=[]
    em_tags = soup.find_all('em')

    for em_tag in em_tags:
        list_of_em_tags.append(em_tag.get_text())
    if len(list_of_em_tags) >=2:
        if is_number(list_of_em_tags[1].split()[0]):
            item_dict['review'] = float(list_of_em_tags[1].split()[0])     
    else:
        item_dict['review']= 0

    
    #Scrape the product ID, sub-category, sub-sub-category, product name, description and price
    item_dict['product_ID']= driver.find_element(By.XPATH,'//*[@id="left"]/section/div[1]/h2/span[2]/em/span/span[2]').text
    if category=='WOMEN':
        item_dict['category'] ='WOMEN'
    else:
        item_dict['category'] ='MEN'
    item_dict['sub_cat']= driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/div/div[1]/ol/li[3]/span').text
    item_dict['ssub_cat']=driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/div/div[1]/ol/li[4]/a').text
    item_dict['name'] = driver.find_element(By.XPATH, '//*[@id="right"]/div[1]/div/div[1]/h1/span').text
    item_dict['price'] = driver.find_element(By.XPATH, '//*[@id="right"]/div[1]/div/div[2]/div[1]/div/div[1]/div/span/span/span').text
    item_dict['desc'] = driver.find_element(By.XPATH, '//*[@id="right"]/div[1]/div/div[3]/div').text
        
    color_sizes={}
    size_avail={}
    col_cnt=1

    #Iterate all the colors until not found. Exception is expected
    #Click on each color and capture the available sizes
    while True:
        try:
            xpath = f'//*[@id="right"]/div[2]/div[1]/div[1]/div/dl/div[{col_cnt}]/dt/label'
            click_element(driver, xpath)
            time.sleep(0.1)
        except:
            break;

        # Scrap the color name
        color_name = driver.find_element(By.XPATH,'//*[@id="right"]/div[2]/div[1]/div[1]/div/h7/span').text

        #load page source into BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)

        # Scrap all the size of the item for mapping use later
        list_of_sizes=[]
        for dl in soup.find_all('dl', class_ ='fr-chips _2jbyX3jUG0P1-2Zuj7ydQv'):
            for el in dl:
                if el.text in ['XXS', 'XS','S','M','L','XL','XXL',\
                       '22 inch', '23 inch','24 inch','25 inch','26 inch','27 inch','28 inch', \
                       '29 inch', '30 inch','31 inch','32 inch','33 inch',  \
                       '34 inch','35 inch','36 inch','37 inch','38 inch',  \
                       '24-26cm', '27-29cm', '27–29 cm', 'S/M', 'L/XL']:
                    list_of_sizes.append(el.text)

        input_element = soup.find_all('input', {'name': 'product-size-picker'})

        i=0
        size_avail={}
        for size in input_element:
            if size.has_attr('disabled'):
                size_avail[list_of_sizes[i]] = 0
            else:
                size_avail[list_of_sizes[i]] = 1
            i+=1

        col_cnt +=1
        color_sizes[color_name] = size_avail.copy()

    item_dict['color_size_avail'] = color_sizes.copy()
    
#---------------------------------------------------------------------------------------------------------------------
all_item={}
item_dict={}
all_item_count=0

categories =['WOMEN','MEN']

no_of_itemperpage =24

sub_cat_women =[1,2,3,4,5,6,7,8,9,12]
sub_cat_men =[1,2,3,4,5,6,7,8,10,11,12]
driver = webdriver.Chrome()
    
# Wait for the page to be loaded and send an"ESC' Keypress to close the popup
time.sleep(5)
pyautogui.press('esc')
time.sleep(2)

for category in categories:

    url = f'https://www.uniqlo.com/ca/en/{category}'
    driver.get(url)
    if category=='WOMEN':
        sub_categoies=sub_cat_women
    else:
        sub_categoies=sub_cat_men
    
    for sub_cat in sub_categoies:
        try:
            time.sleep(3)
            pyautogui.press('esc')
            print(f'category : {category} , sub_cat :{sub_cat}')
            
            # Prepare the XPATH for sub-cat and click inside sub-categort afterward
            xpath = f'//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/nav[2]/div/div[1]/article[{sub_cat}]/a/span'
  
            # Click inside the sub-cat
            click_element(driver, xpath)
            #WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()

            time.sleep(5)
            pyautogui.press('esc')
    
            #Get the no of item in the sub-cat
            no_of_item = get_no_of_item()
            
            #Calculate the number of click required on 'LOAD MORE' button to show all items
            num_of_loadmore = no_of_item//no_of_itemperpage -1
            if no_of_item%no_of_itemperpage != 0:
                num_of_loadmore +=1
            
            #Show all items in the sub-cat
            show_all_items(num_of_loadmore)
    
        except Exception as e:
            print("Exception #1: will skip the whole category")
            continue
            print(e)
        
        try:

            for item_no in range(no_of_item):    
                # Prepare the XPATH for each item
                if category == 'MEN' and sub_cat==3:
                    xpath=f'//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/section/div/div/div[1]/div[3]/div/div/article[{item_no+1}]/a/div/div[1]/div/img'
                elif category == 'MEN' and sub_cat==5:
                    xpath=f'//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/section[4]/div/div/div[1]/div[3]/div/div/article[{item_no+1}]/a/div/div[1]/div/img'
                else:
                    xpath=f'//*[@id="root"]/div/div/div/div/main/div/div[2]/div/div/div/section[3]/div/div/div[1]/div[3]/div/div/article[{item_no+1}]/a'
               
                pyautogui.press('esc')
                click_element(driver, xpath)
                time.sleep(5)
                
                # load the page souce into BeautifulSoup
                page_source = driver.page_source
                soup = BeautifulSoup(page_source)
                pyautogui.press('esc')
                
                # Scrape the detail of the item
                try :
                    get_item_details()
                
                except Exception as e:
                    print('exception #3')
                    print(e)
                    continue
                # Copy the item detail to item list
                all_item[all_item_count+1]=item_dict.copy()
                print(f'item_dict : {item_dict} \n  all_item_count : {all_item_count} \n')
                all_item_count +=1
                driver.back()
            
        except Exception as e:
            print("Exception #2")
            print(e)
            
        driver.back()